<a href="https://colab.research.google.com/github/Ravi-shankar100/CNN/blob/master/MnistFashionOptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [ ]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [18]:
print(tf.__version__)

2.2.0


In [19]:
fashion_mnist=keras.datasets.fashion_mnist

In [20]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [21]:
train_images=train_images/255.0
test_images=test_images/255.0

In [22]:
train_images.shape

(60000, 28, 28)

In [23]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [26]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [27]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3949 - accuracy: 0.8566 - val_loss: 0.2918 - val_accuracy: 0.8957
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2566 - accuracy: 0.9051 - val_loss: 0.2598 - val_accuracy: 0.9045
Epoch 3/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.2020 - accuracy: 0.9240 - val_loss: 0.2549 - val_accuracy: 0.9103


Epoch 1/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.3807 - accuracy: 0.8631 - val_loss: 0.2836 - val_accuracy: 0.8922
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2397 - accuracy: 0.9106 - val_loss: 0.2518 - val_accuracy: 0.9092
Epoch 3/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.1753 - accuracy: 0.9350 - val_loss: 0.2275 - val_accuracy: 0.9223


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4476 - accuracy: 0.8419 - val_loss: 0.3400 - val_accuracy: 0.8725
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3240 - accuracy: 0.8800 - val_loss: 0.3229 - val_accuracy: 0.8762
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2805 - accuracy: 0.8953 - val_loss: 0.3579 - val_accuracy: 0.8750


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3685 - accuracy: 0.8666 - val_loss: 0.2883 - val_accuracy: 0.8927
Epoch 2/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2265 - accuracy: 0.9168 - val_loss: 0.2511 - val_accuracy: 0.9067
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1587 - accuracy: 0.9404 - val_loss: 0.2580 - val_accuracy: 0.9098


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3756 - accuracy: 0.8655 - val_loss: 0.2933 - val_accuracy: 0.8925
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2304 - accuracy: 0.9162 - val_loss: 0.2430 - val_accuracy: 0.9127
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1650 - accuracy: 0.9386 - val_loss: 0.2349 - val_accuracy: 0.9167


INFO:tensorflow:Oracle triggered exit


In [29]:
model= tuner_search.get_best_models(num_models= 1)[0]

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,700,074
Trainable params: 2,700,074
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(train_images,train_labels,epochs= 10,validation_split= 0.1,initial_epoch= 2)

Epoch 3/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1269 - accuracy: 0.9520 - val_loss: 0.2631 - val_accuracy: 0.9150
Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0888 - accuracy: 0.9664 - val_loss: 0.2960 - val_accuracy: 0.9173
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0616 - accuracy: 0.9774 - val_loss: 0.3257 - val_accuracy: 0.9183
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0416 - accuracy: 0.9844 - val_loss: 0.3678 - val_accuracy: 0.9153
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0338 - accuracy: 0.9876 - val_loss: 0.4574 - val_accuracy: 0.9115
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0282 - accuracy: 0.9894 - val_loss: 0.4522 - val_accuracy: 0.9070
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0240 - accuracy: 0.9915 - val_loss: 0.4653 - val_accuracy: